In [16]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.convolutional import Conv1D, MaxPooling1D
import keras.utils as ku 
from tensorflow.keras.optimizers import Adam, RMSprop

# set seeds for reproducability
from numpy.random import seed
seed(1)

import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
import re

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/game-of-thrones-dataset-text-generation/got1.txt


## 1. Load the dataset

In [2]:
# read the files 
def read_file(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

data = read_file('/kaggle/input/game-of-thrones-dataset-text-generation/got1.txt')

In [5]:
data[:100]

'A Game Of Thrones \nBook One of A Song of Ice and Fire \nBy George R. R. Martin \nPROLOGUE \n"We should '

In [9]:
# total length of the text dataset
len(data)

1607894

## 2. Dataset preparation

In [18]:
# clean the data 
def cleaning(text):
    sample = text
    sample = re.sub('[%s]' % re.escape(string.punctuation), '', sample)
    sample = [word for word in sample.split() if word.isalpha()]
    sample = [word.lower() for word in sample]
    sample = " ".join(sample)
    return sample

In [20]:
# return the cleaned data
cleaned_data = cleaning(data)
len(cleaned_data)

1512606

In [21]:
# check out first 100 words
cleaned_data[:100]

'a game of thrones book one of a song of ice and fire by george r r martin prologue we should start b'

In [23]:
# number of words in cleaned data
words = [word for word in cleaned_data.split()]
print("Total number of words:",len(words))

Total number of words: 292883


In [26]:
# let's also print unique words
uniq_words = set(words)
print("Total unique words:", len(u+niq_words))

Total unique words: 11923
